In [465]:
import pandas as pd
import pprint
from messari.messari import Messari
from tabulate import tabulate


In [466]:
messari = Messari("56d86702-1c8f-4d13-b3a3-971f50366cab")

assetsData = messari.get_all_assets(limit=500)["data"]

In [467]:
columns = list(pd.DataFrame(assetsData).columns)
metrics = list(assetsData[0]["metrics"].keys())
profile = list(assetsData[0]["profile"].keys())

metricsKeys = {}
profileKeys = {}

for i in range(len(metrics)):
    try:
        metricsKeys[metrics[i]] = list(assetsData[0]["metrics"][metrics[i]].keys())
    except:
        pass

for i in range(len(profile)):
    try:
        profileKeys[profile[i]] = list(assetsData[0]["profile"][profile[i]].keys())
    except:
        pass



print(columns)
print("\n")
print(metricsKeys.keys())
print("\n")
print(profileKeys.keys())
#print("\n")
#print(symbols)

['id', 'serial_id', 'symbol', 'name', 'slug', 'contract_addresses', '_internal_temp_agora_id', 'metrics', 'profile']


dict_keys(['market_data', 'marketcap', 'supply', 'blockchain_stats_24_hours', 'market_data_liquidity', 'all_time_high', 'cycle_low', 'token_sale_stats', 'mining_stats', 'developer_activity', 'roi_data', 'roi_by_year', 'risk_metrics', 'misc_data', 'reddit', 'on_chain_data', 'exchange_flows', 'miner_flows', 'supply_activity', 'supply_distribution'])


dict_keys(['general', 'contributors', 'advisors', 'investors', 'ecosystem', 'economics', 'technology', 'governance', 'metadata'])


In [468]:
masterlist = []

useMetricKeys = ["market_data", "marketcap", "supply", "token_sale_stats", "developer_activity", "roi_data", "roi_by_year", "risk_metrics", "supply_activity", "supply_distribution"]
useProfKeys = ['economics', 'governance']
# for each asset
for asset in range(len(assetsData)):
    assetdict = {}
    assetdict["name"] =  assetsData[asset]["name"]
    assetdict["symbol"] = assetsData[asset]["symbol"]

    # for each metric in the asset
    for metKey in metricsKeys:
        # if metric is in the list of metrics to use
        if metKey in useMetricKeys:
            # for each column within the metric key
            for k in metricsKeys[metKey]:
                # if column itself has a list of values 
                if type(assetsData[asset]["metrics"][metKey][k]) == dict:
                    # separate the list of values into separate columns
                    for dk in assetsData[asset]["metrics"][metKey][k]:
                        assetdict[k+"__"+dk] = assetsData[asset]["metrics"][metKey][k][dk]
                # otherwise store as column value
                else:
                    assetdict[k] = assetsData[asset]["metrics"][metKey][k]
        else:
            pass

    # for each profile in the asset
    for profKey in profileKeys:
        # if profile is in the list of profiles to use
        if profKey in useProfKeys:
            # for each column within the profile key
            for k in profileKeys[profKey]:
                if k in ["grants"]:
                    pass
                else:
                    # if column itself has a list of values 
                    if type(assetsData[asset]["profile"][profKey][k]) == dict:
                        # separate the list of values into separate columns
                        for dk in assetsData[asset]["profile"][profKey][k]:
                            # if profile:economics:block_explorer don't bother
                            if dk in ["block_explorers", "accounts", "multitoken"]:
                                pass
                            else:
                                #if recursive dictionary do it again
                                if type(assetsData[asset]["profile"][profKey][k][dk]) == dict:
                                    for ddk in assetsData[asset]["profile"][profKey][k][dk]:
                                        assetdict[dk+"__"+ddk] = assetsData[asset]["profile"][profKey][k][dk][ddk]
                                elif type(assetsData[asset]["profile"][profKey][k][dk]) == list:
                                    if len(assetsData[asset]["profile"][profKey][k][dk]) == 0:
                                        assetdict[k+"__"+dk] = None
                                    else:
                                        print(k+"__"+dk, assetsData[asset]["profile"][profKey][k][dk])
                                else:
                                    assetdict[k+"__"+dk] = assetsData[asset]["profile"][profKey][k][dk]

                    # otherwise store as column value
                    elif type(assetsData[asset]["profile"][profKey][k]) == list:
                        if len(assetsData[asset]["profile"][profKey][k]) == 0:
                            assetdict[k] = None
                        else:
                            pass
                    else:
                        assetdict[k] = assetsData[asset]["profile"][profKey][k]
        else:
            pass
    
    # append in master list
    masterlist.append(assetdict)

# store as dataframe
assetdf = pd.DataFrame(masterlist)
assetdf

,name,symbol,price_usd,price_btc,price_eth,volume_last_24_hours,real_volume_last_24_hours,volume_last_24_hours_overstatement_multiple,percent_change_usd_last_1_hour,percent_change_btc_last_1_hour,...,consensus__mining_algorithm,consensus__next_halving_date,consensus__is_victim_of_51_percent_attack,native_treasury__treasury_usage_details,governance_details,onchain_governance__onchain_governance_type,onchain_governance__onchain_governance_details,onchain_governance__is_treasury_decentralized,ohlcv_last_1_hour,ohlcv_last_24_hour
0,Bitcoin,BTC,38788.140970,1.000000e+00,13.665409,4.859843e+09,2.895187e+09,1.678594,-0.360287,0.000000,...,SHA-256,2024-05-10T04:00:00Z,False,None,Model Overview\nBitcoin governance is the proc...,None,None,False,NaN,NaN
1,Ethereum,ETH,2838.422536,7.317759e-02,1.000000,2.986098e+09,1.789742e+09,1.668452,-0.631491,-0.272184,...,Ethash,None,False,None,Ethereum operates similarly to Bitcoin through...,No On-Chain Governance,While Ethereum does not have an official on-ch...,False,NaN,NaN
2,Tether,USDT,0.999711,2.577363e-05,0.000352,1.106205e+08,8.166795e+07,1.354516,-0.003574,0.358003,...,None,None,None,None,Tether Limited is the sole entity capable of a...,None,None,None,NaN,NaN
3,BNB,BNB,388.634977,1.001943e-02,0.136919,1.664320e+08,1.171148e+08,1.421102,-0.215474,0.145337,...,n/a,None,False,None,"Currently, BNB is governed in a centralized ma...",Upcoming,Parameters for the governance module are outli...,False,NaN,NaN
4,USD Coin,USDC,1.000371,2.579065e-05,0.000352,9.942964e+08,9.708380e+08,1.024163,0.049502,0.411271,...,None,None,None,None,CENTRE members are the sole entities capable o...,None,None,None,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,TrueChain,TRUE,0.040351,1.040519e-06,0.000014,7.274673e+04,7.274673e+04,1.000000,NaN,NaN,...,n/a,None,None,None,None,None,None,None,NaN,NaN
496,Eden Network,EDEN,0.236210,6.057241e-06,0.000083,3.721354e+05,3.721354e+05,1.000000,NaN,NaN,...,None,None,None,None,High level project governance is currently the...,None,None,None,NaN,NaN
497,Razor Network,RAZOR,0.013271,3.418380e-07,0.000005,1.142144e+04,1.142144e+04,1.000000,NaN,NaN,...,None,None,None,None,None,None,None,None,NaN,NaN
498,Matrix,MAN,0.015271,3.939745e-07,0.000005,7.416676e+04,6.441985e+04,1.151303,2.584926,2.988125,...,None,None,None,None,None,None,None,None,NaN,NaN


In [470]:
# write to csv as well as plaintext
assetdf.to_csv("assets.csv")
with open('table.txt', 'w') as f:
    f.write(tabulate(assetdf, headers='keys', tablefmt='grid'))